In [25]:
import pandas as pd
import re
from gensim import corpora,models,similarities
from gensim.utils import tokenize


In [26]:
df = pd.read_csv('base.csv',sep=",",names = ["names","text"])[["names", "text"]]

In [27]:
df

,names,text
0,./repos/tmate/tmate-msg.c,static void tmate_status_message_client(struct...
1,./repos/tmate/tmate-msg.c,static void tmate_status_message_session(const...
2,./repos/tmate/tmate-msg.c,"void __tmate_status_message(const char *fmt, v..."
3,./repos/tmate/tmate-msg.c,"void tmate_status_message(const char *fmt, ......"
4,./repos/tmate/cmd-queue.c,struct cmd_q *\ncmdq_new(struct client *c)\n{\...
...,...,...
66,./repos/tmate/utf8.c,char *\nutf8_tocstr(struct utf8_data *src)\n{\...
67,./repos/tmate/utf8.c,u_int\nutf8_cstrwidth(const char *s)\n{\n\tstr...
68,./repos/tmate/utf8.c,"char *\nutf8_trimcstr(const char *s, u_int wid..."
69,./repos/tmate/utf8.c,"char *\nutf8_rtrimcstr(const char *s, u_int wi..."


In [28]:
df_1 = pd.read_csv('to_search_for.csv',sep=",",names = [1,"text"])[["names", "text"]]

In [29]:
list(df.head(5)["text"].values)
list(df_1.head(5)["text"].values)

["int strvis(undefined2 *param_1,char *param_2,uint param_3)\n\n{\n  char *pcVar1;\n  char cVar2;\n  undefined2 *puVar3;\n  \n  cVar2 = *param_2;\n  puVar3 = param_1;\n  if (cVar2 == '\\0') {\n    *(undefined *)param_1 = 0;\n    return 0;\n  }\n  do {\n    pcVar1 = param_2 + 1;\n    param_2 = param_2 + 1;\n    puVar3 = vis(puVar3,(int)cVar2,param_3,*pcVar1);\n    cVar2 = *param_2;\n  } while (cVar2 != '\\0');\n  *(undefined *)puVar3 = 0;\n  return (int)puVar3 - (int)param_1;\n}",
 'void alerts_ring_bell(undefined4 param_1,undefined4 param_2,undefined4 param_3,undefined4 param_4,\n                     undefined4 param_5,undefined4 param_6,undefined4 param_7,undefined4 param_8,\n                     long param_9,undefined8 param_10,undefined8 param_11,undefined8 param_12,\n                     undefined8 param_13,undefined8 param_14)\n\n{\n  long *plVar1;\n  undefined8 extraout_RDX;\n  long lVar2;\n  \n  lVar2 = clients;\n  if (clients != 0) {\n    do {\n      while ((*(long *)(lVar2 + 0

In [30]:
texts_to_compare = list(df_1["text"])

In [31]:
def tokenize_in_df(strin):
    try:
        return list(tokenize(strin,))
    except:
        return ""

In [32]:
df["tokens"] = df["text"].apply(tokenize_in_df)

In [33]:
df

,names,text,tokens
0,./repos/tmate/tmate-msg.c,static void tmate_status_message_client(struct...,"[static, void, tmate_status_message_client, st..."
1,./repos/tmate/tmate-msg.c,static void tmate_status_message_session(const...,"[static, void, tmate_status_message_session, c..."
2,./repos/tmate/tmate-msg.c,"void __tmate_status_message(const char *fmt, v...","[void, __tmate_status_message, const, char, fm..."
3,./repos/tmate/tmate-msg.c,"void tmate_status_message(const char *fmt, ......","[void, tmate_status_message, const, char, fmt,..."
4,./repos/tmate/cmd-queue.c,struct cmd_q *\ncmdq_new(struct client *c)\n{\...,"[struct, cmd_q, cmdq_new, struct, client, c, s..."
...,...,...,...
66,./repos/tmate/utf8.c,char *\nutf8_tocstr(struct utf8_data *src)\n{\...,"[char, utf, _tocstr, struct, utf, _data, src, ..."
67,./repos/tmate/utf8.c,u_int\nutf8_cstrwidth(const char *s)\n{\n\tstr...,"[u_int, utf, _cstrwidth, const, char, s, struc..."
68,./repos/tmate/utf8.c,"char *\nutf8_trimcstr(const char *s, u_int wid...","[char, utf, _trimcstr, const, char, s, u_int, ..."
69,./repos/tmate/utf8.c,"char *\nutf8_rtrimcstr(const char *s, u_int wi...","[char, utf, _rtrimcstr, const, char, s, u_int,..."


In [34]:
dictionary = corpora.Dictionary(df["tokens"])

In [35]:
dictionary.token2id

{'CLIENT_FORCE_STATUS': 0,
 'CLIENT_STATUS': 1,
 'L': 2,
 'NULL': 3,
 'TAILQ_FOREACH_SAFE': 4,
 'TAILQ_INSERT_TAIL': 5,
 'TAILQ_REMOVE': 6,
 'break': 7,
 'c': 8,
 'char': 9,
 'client': 10,
 'const': 11,
 'delay': 12,
 'display': 13,
 'entry': 14,
 'event_initialized': 15,
 'evtimer_add': 16,
 'evtimer_del': 17,
 'evtimer_set': 18,
 'flags': 19,
 'free': 20,
 'global_options': 21,
 'global_s_options': 22,
 'if': 23,
 'int': 24,
 'limit': 25,
 'message': 26,
 'message_entry': 27,
 'message_log': 28,
 'message_next': 29,
 'message_string': 30,
 'message_timer': 31,
 'msg': 32,
 'msg_num': 33,
 'msg_time': 34,
 'options': 35,
 'options_get_number': 36,
 'recalculate_sizes': 37,
 's': 38,
 'session': 39,
 'sizeof': 40,
 'static': 41,
 'status_message_callback': 42,
 'status_message_clear': 43,
 'status_prompt_clear': 44,
 'struct': 45,
 'time': 46,
 'timeval': 47,
 'tmate': 48,
 'tmate_status_message_client': 49,
 'tv': 50,
 'tv_sec': 51,
 'tv_usec': 52,
 'u_int': 53,
 'void': 54,
 'xasprin

In [36]:
feature_cnt = len(dictionary.token2id)

In [37]:
feature_cnt

528

In [38]:
corpus = [dictionary.doc2bow(text) for text in df["tokens"]]

In [39]:
corpus[0]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 18),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 5),
 (13, 1),
 (14, 3),
 (15, 1),
 (16, 1),
 (17, 1),
 (18, 1),
 (19, 1),
 (20, 2),
 (21, 1),
 (22, 1),
 (23, 3),
 (24, 1),
 (25, 4),
 (26, 3),
 (27, 1),
 (28, 3),
 (29, 2),
 (30, 2),
 (31, 4),
 (32, 16),
 (33, 2),
 (34, 1),
 (35, 1),
 (36, 2),
 (37, 1),
 (38, 1),
 (39, 2),
 (40, 1),
 (41, 1),
 (42, 1),
 (43, 1),
 (44, 1),
 (45, 3),
 (46, 2),
 (47, 1),
 (48, 2),
 (49, 1),
 (50, 4),
 (51, 1),
 (52, 1),
 (53, 1),
 (54, 1),
 (55, 1),
 (56, 1),
 (57, 1)]

In [40]:
tfidf = models.TfidfModel(corpus)

In [41]:
index = similarities.SparseMatrixSimilarity(tfidf[corpus],num_features = feature_cnt)

In [42]:
for text in texts_to_compare:
    kw_vector = dictionary.doc2bow(tokenize(text))
    df[text] = index[tfidf[kw_vector]]

In [49]:
df = df.sort_values(by=df_1["text"][0], ascending=False)

In [50]:
df

,names,text,tokens,"int strvis(undefined2 *param_1,char *param_2,uint param_3)\n\n{\n char *pcVar1;\n char cVar2;\n undefined2 *puVar3;\n \n cVar2 = *param_2;\n puVar3 = param_1;\n if (cVar2 == '\0') {\n *(undefined *)param_1 = 0;\n return 0;\n }\n do {\n pcVar1 = param_2 + 1;\n param_2 = param_2 + 1;\n puVar3 = vis(puVar3,(int)cVar2,param_3,*pcVar1);\n cVar2 = *param_2;\n } while (cVar2 != '\0');\n *(undefined *)puVar3 = 0;\n return (int)puVar3 - (int)param_1;\n}","void alerts_ring_bell(undefined4 param_1,undefined4 param_2,undefined4 param_3,undefined4 param_4,\n undefined4 param_5,undefined4 param_6,undefined4 param_7,undefined4 param_8,\n long param_9,undefined8 param_10,undefined8 param_11,undefined8 param_12,\n undefined8 param_13,undefined8 param_14)\n\n{\n long *plVar1;\n undefined8 extraout_RDX;\n long lVar2;\n \n lVar2 = clients;\n if (clients != 0) {\n do {\n while ((*(long *)(lVar2 + 0x598) != param_9 || ((*(byte *)(lVar2 + 0x421) & 0x20) != 0))) {\n plVar1 = (long *)(lVar2 + 0x5c0);\n lVar2 = *plVar1;\n if (*plVar1 == 0) {\n return;\n }\n }\n param_1 = tty_putcode(param_1,param_2,param_3,param_4,param_5,param_6,param_7,param_8,\n lVar2 + 0xe0,3,param_11,param_12,param_13,param_14);\n plVar1 = (long *)(lVar2 + 0x5c0);\n param_11 = extraout_RDX;\n lVar2 = *plVar1;\n } while (*plVar1 != 0);\n }\n return;\n}"
63,./repos/tmate/utf8.c,"int\nutf8_strvis(char *dst, const char *src, s...","[int, utf, _strvis, char, dst, const, char, sr...",0.135378,0.015672
21,./repos/tmate/proc.c,"void\nproc_loop(struct tmuxproc *tp, int (*loo...","[void, proc_loop, struct, tmuxproc, tp, int, l...",0.116586,0.041602
37,./repos/tmate/alerts.c,"int\nalerts_enabled(struct window *w, int flag...","[int, alerts_enabled, struct, window, w, int, ...",0.076210,0.012374
19,./repos/tmate/proc.c,"int\nproc_send_s(struct tmuxpeer *peer, enum m...","[int, proc_send_s, struct, tmuxpeer, peer, enu...",0.070120,0.002039
67,./repos/tmate/utf8.c,u_int\nutf8_cstrwidth(const char *s)\n{\n\tstr...,"[u_int, utf, _cstrwidth, const, char, s, struc...",0.066991,0.105414
...,...,...,...,...,...
49,./repos/tmate/arguments.c,void\nargs_free(struct args *args)\n{\n\tstruc...,"[void, args_free, struct, args, args, struct, ...",0.000000,0.000673
56,./repos/tmate/utf8.c,"void\nutf8_set(struct utf8_data *ud, u_char ch...","[void, utf, _set, struct, utf, _data, ud, u_ch...",0.000000,0.000661
13,./repos/tmate/cmd-queue.c,void\ncmdq_flush(struct cmd_q *cmdq)\n{\n\tstr...,"[void, cmdq_flush, struct, cmd_q, cmdq, struct...",0.000000,0.000617
29,./repos/tmate/window-clock.c,void\nwindow_clock_resize(struct window_pane *...,"[void, window_clock_resize, struct, window_pan...",0.000000,0.000881


In [52]:
for text in df_1["text"]:
    print(text[0])
    df = df.sort_values(by=text, ascending=False)
    print(df.loc[df[text]>0.150]["names"])

int strvis(undefined2 *param_1,char *param_2,uint param_3)

{
  char *pcVar1;
  char cVar2;
  undefined2 *puVar3;
  
  cVar2 = *param_2;
  puVar3 = param_1;
  if (cVar2 == '\0') {
    *(undefined *)param_1 = 0;
    return 0;
  }
  do {
    pcVar1 = param_2 + 1;
    param_2 = param_2 + 1;
    puVar3 = vis(puVar3,(int)cVar2,param_3,*pcVar1);
    cVar2 = *param_2;
  } while (cVar2 != '\0');
  *(undefined *)puVar3 = 0;
  return (int)puVar3 - (int)param_1;
}
Series([], Name: names, dtype: object)
void alerts_ring_bell(undefined4 param_1,undefined4 param_2,undefined4 param_3,undefined4 param_4,
                     undefined4 param_5,undefined4 param_6,undefined4 param_7,undefined4 param_8,
                     long param_9,undefined8 param_10,undefined8 param_11,undefined8 param_12,
                     undefined8 param_13,undefined8 param_14)

{
  long *plVar1;
  undefined8 extraout_RDX;
  long lVar2;
  
  lVar2 = clients;
  if (clients != 0) {
    do {
      while ((*(long *)(lVar2 + 0x59

In [ ]:
df["tokens"][4]

In [ ]:
df.loc[df["sum"]>0.250]["names"]

In [ ]:
df[df["sum"]>0.250].to_excel("похожие тексты.xlsx")

In [14]:
import csv
import os

with open("repos-test/alerts.c", 'r') as file:
    f = file.read()
    new_f = f.replace('"', '""')

In [15]:
new_f

'/* $OpenBSD$ */\n\n/*\n * Copyright (c) 2015 Nicholas Marriott <nicholas.marriott@gmail.com>\n *\n * Permission to use, copy, modify, and distribute this software for any\n * purpose with or without fee is hereby granted, provided that the above\n * copyright notice and this permission notice appear in all copies.\n *\n * THE SOFTWARE IS PROVIDED ""AS IS"" AND THE AUTHOR DISCLAIMS ALL WARRANTIES\n * WITH REGARD TO THIS SOFTWARE INCLUDING ALL IMPLIED WARRANTIES OF\n * MERCHANTABILITY AND FITNESS. IN NO EVENT SHALL THE AUTHOR BE LIABLE FOR\n * ANY SPECIAL, DIRECT, INDIRECT, OR CONSEQUENTIAL DAMAGES OR ANY DAMAGES\n * WHATSOEVER RESULTING FROM LOSS OF MIND, USE, DATA OR PROFITS, WHETHER\n * IN AN ACTION OF CONTRACT, NEGLIGENCE OR OTHER TORTIOUS ACTION, ARISING\n * OUT OF OR IN CONNECTION WITH THE USE OR PERFORMANCE OF THIS SOFTWARE.\n */\n\n#include <sys/types.h>\n\n#include <event.h>\n\n#include ""tmux.h""\n\nint\talerts_fired;\n\nvoid\talerts_timer(int, short, void *);\nint\talerts_ena